# モデルを考察する（３）

より単純なモデルを対象に、振動の仕組みを考えてみる。

## 準備作業

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import biosim_course

# FitzHugh-Nagumo モデル

1960年代に発表された古典的な神経細胞のモデル。

![Model](./Lec-12-FNmodel.png)


* 変数は _u_, _v_ の２つだけ。
* **_u_** は膜電位、**_v_** は膜電位の不活性化の程度を表す。_I_ は細胞外から流入する外部電流。_a_, _b_, _c_ はモデルパラメータ。
* _u_, _v_ が相互に刺激・抑制することで、周期振動が起こる。


## シミュレーションを実行し、_u_, _v_ の動きをグラフに描く

In [ ]:
EM = '''
Stepper ODEStepper( ODE ) {}

System System( / )
{
    StepperID  ODE;

    Variable Variable( u ) { Value  0.0; }
    Variable Variable( v ) { Value  0.0; }
    
    Process ExpressionFluxProcess( dudt )
    {
        c  10.0;
        I  0.5;
        Expression  "c * ( -v.Value + u.Value - pow( u.Value, 3.0 )/3.0 + I )";
        VariableReferenceList
            [ u   :.:u  1 ]
            [ v   :.:v  0 ];
    }
    
    Process ExpressionFluxProcess( dvdt )
    {
        a  0.7;
        b  0.8;
        Expression  "u.Value - b * v.Value + a";
        VariableReferenceList
            [ u   :.:u  0 ]
            [ v   :.:v  1 ];
    }
}
'''

In [ ]:
setModel( EM, "Tyson1991")
print 't = {}'.format( getCurrentTime() )

### システムパス `/` 内のすべての `Variable`, `Process` の `LoggerStub` を作成し、`create()` する

In [ ]:
target_SystemPath_list = ( '/', )
Target_Properties = { 'Variable': ['Value', 'Velocity'], 'Process': ['Activity'] }
Logger_dict = {}

for target_SystemPath in target_SystemPath_list:
    for E_type, Properties in Target_Properties.items():
        for E in getEntityList( E_type, target_SystemPath ):
            for p in Properties:
                FullPN = ':'.join( ( E_type, target_SystemPath, E, p ) )
                Logger_dict[ FullPN ] = createLoggerStub( FullPN )
                Logger_dict[ FullPN ].create()

In [ ]:
step_width = 10.0  # min

run( step_width )
print 't = {}'.format( getCurrentTime() )

In [ ]:
Data_dict = {}
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 0, getCurrentTime(), getCurrentTime() / 100 ) )[ :, :2 ]

plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN.split(':')[3] == 'Value':
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

## _u, v_ の関係を可視化するために、相空間プロットを描く

* 相空間プロットとは、グラフのそれぞれの軸に変数を割り当て、変数と変数の関係を直接グラフに描いたもの。
* 横軸が時間でなくなっているので、時間の情報が失われる。


In [ ]:
u_Data = Data_dict[ 'Variable:/:u:Value' ][ :, 1 ]
v_Data = Data_dict[ 'Variable:/:v:Value'  ][ :, 1 ]

plt.figure()
plt.plot( u_Data, v_Data )
plt.xlabel( 'u' )
plt.ylabel( 'v' )
plt.xlim( u_Data.min() - ( ( u_Data.max() - u_Data.min() ) * 0.1 ), u_Data.max() + ( ( u_Data.max() - u_Data.min() ) * 0.1 ) )
plt.ylim( v_Data.min() - ( ( v_Data.max() - v_Data.min() ) * 0.1 ), v_Data.max() + ( ( v_Data.max() - v_Data.min() ) * 0.1 ) )


* 周期的に振動している系を相空間プロットすると、このような軌道が現れる。
* このモデルの場合、_u, v_ は、軌道を反時計回りに周回している。


## 振動をいくつかのフェイズ（相）に分けて、動態を考察する

* ほぼ直線状の各辺では、一定の力関係で系が変化していると推定される。
* コーナーを回るところで、_u_, _v_ の関係に変化が起こり、別の力関係へとシフトしていると思われる。
* そこで、１周期を四角形の辺にあたる４つの区間に分け、各区間とその移行期に何が起こっているのかを個別に考えてみる。


### 微分方程式を「読む」

* このモデルは、以下の２つの微分方程式からなる。
    * _du/dt_ は、 _u_ の変化速度の意味（ _u_ の時間（ _t_ ）微分）。

$$
\begin{eqnarray}
  \frac{ du }{ dt } = c \left( -v + u -  \frac{ u^3 }{ 3 } + I \right) \\
  \frac{ dv }{ dt } = u - bv + a
\end{eqnarray}
$$


* モデル中で、時間とともに値が変化するもの（変数）は、 _u_ , _v_ の２つだけ。



### 微分方程式を分解して、２つの変数の関係をまとめる

* ２つの変数が、お互いにどういった影響を及ぼしあっているか、微分方程式を分解して表にまとめてみる。

![Analysis](./Lec-12-FN-table.png)

* _u_ が _u_ 自身に与える影響以外は単純。
* _v_ が与える影響については、 _c_ = 10.0, _b_ = 0.8 なので、_v_ が自分自身を抑制する影響より、 _u_ を抑制する影響の方が 10 倍以上強いことがわかる。
    * _v_ が負の時には刺激に変わるので、ここで **抑制** と読んでいる作用は **対象をゼロに近づける作用** と考えるとよい。


### _u_ が自分自身に与える効果を可視化する

* _u_ が自分自身に与える効果がやや複雑なので、グラフに描いて視覚的に把握しておく。
* _u_ が _u_ 自身に与える効果（u - u^3/3）をグラフに描いてみる。
    * _u_ が _v_ に与える効果も一緒に描き、その大小関係を比較する。


In [ ]:
x = np.arange( -2.0, 2.0, 0.1 )
plt.figure()
plt.plot( x, 10 * ( x - x**3/3 ), label = 'u -> u' )
plt.plot( x, x, label = 'u -> v' )
plt.legend( loc = 'lower right' )
plt.xlabel( 'u' )
plt.ylabel( 'effect of u (min^-1)' )

* _u_ は -√3 ＜ _u_ ＜ 0 および √3 ＜ _u_ の範囲で負の値をとる。すなわち、_u_ 自身を抑制する（ネガティブフィードバック）。  
それ以外の領域ではゼロまたは正の値をとり、_u_ 自身を刺激する（ポジティブフィードバック）。
* _u_ は 0 ＜ _u_ のとき _v_ を促進し、_u_ ＜ 0 のとき _v_ を抑制する。
* _v_ は 0 ＜ _v_ のとき _u_ を抑制し、_v_ ＜ 0 のとき _u_ を促進する。
* _v_ は 0 ＜ _v_ のとき _v_ 自身を抑制し、_v_ ＜ 0 のとき _v_ 自身を促進する。


![Analysis](./Lec-12-FN-analysis.png)


1. **_u_ は増加。_v_ は減少**  
_v_ による不活性化が減少することで、_u_ が -2.0 付近から徐々に増加していく。  
_v_ がゼロをやや下回ったところで、_v_ の変化速度がゼロになり、以降、増加に転じる。
2. **_u_, _v_ ともに増加**  
_u_ = -1.0 で、_u_ による自身への抑制は最大となり、これを超えると、抑制が減少し、_u_ = 0 を超えたところで自己刺激（ポジティブフィードバック）に転じ、_u_ は爆発的に増加する。_u_ の自己刺激は、_u_ = 1.0 で極大となる。  
また、_u_ の増加により、_v_ が刺激され、増加に転じる。
3. **_u_ は減少。_v_ は増加**  
_u_ による自己刺激は _u_ = √3 でゼロになり、この付近で、増加した _v_ による不活性化が _u_ の増加（自己刺激および _I_ ）を上回り、_u_ は減少に転じる。  
その際、_u_ = 1.0 付近では、_u_ による自己刺激で、減少に歯止めがかかるが、踏みこらえられずに _u_ の減少はつづく。
4. **_u_, _v_ ともに減少**  
_u_ ＜ 0 となることで、_u_ の自身への効果は抑制に転じ、_u_ は急激に減少する。  
このとき _u_ は、_v_ も抑制するので、_v_ も減少に転じる。  
_u_ = -√3 で _u_ による自己抑制はゼロになり、この付近で _u_ の減少は止まり、増加に転じる。